In [ ]:
#default_exp data.seq2seq.translation
#default_cls_lvl 3

In [ ]:
#all_slow

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# data.seq2seq.translation

> This module contains the bits required to use the fastai DataBlock API and/or mid-level data processing pipelines to organize your data for translation tasks

In [ ]:
#export
import ast
from functools import reduce

import torch
from datasets import list_datasets, load_dataset
from transformers import *
from fastai.text.all import *

from blurr.utils import *
from blurr.data.core import *
from blurr.data.seq2seq.core import *

logging.set_verbosity_error()

In [ ]:
#hide
import pdb

from nbdev.showdoc import *
from fastcore.test import *

from fastai import __version__ as fa_version
from torch import __version__ as pt_version
from transformers import __version__ as hft_version

print(f'Using pytorch {pt_version}')
print(f'Using fastai {fa_version}')
print(f'Using transformers {hft_version}')

Using pytorch 1.7.1
Using fastai 2.3.1
Using transformers 4.6.1


In [ ]:
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

Using GPU #1: GeForce GTX 1080 Ti


In [ ]:
ds = load_dataset('wmt16', 'de-en', split='train[:1%]')

Reusing dataset wmt16 (/home/wgilliam/.cache/huggingface/datasets/wmt16/de-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a)


## Translation tokenization, batch transform, and DataBlock methods

Translation tasks attempt to convert text in one language into another

In [ ]:
path = Path('./')
wmt_df = pd.DataFrame(ds['translation'], columns=['de', 'en']); len(wmt_df)

45489

In [ ]:
wmt_df.head(2)

,de,en
0,Wiederaufnahme der Sitzungsperiode,Resumption of the session
1,"Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.","I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period."


In [ ]:
pretrained_model_name = "facebook/bart-large-cnn"
model_cls = AutoModelForSeq2SeqLM

hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, model_cls=model_cls)
hf_arch, type(hf_tokenizer), type(hf_config), type(hf_model)

('bart',
 transformers.models.bart.tokenization_bart_fast.BartTokenizerFast,
 transformers.models.bart.configuration_bart.BartConfig,
 transformers.models.bart.modeling_bart.BartForConditionalGeneration)

In [ ]:
blocks = (HF_Seq2SeqBlock(hf_arch, hf_config, hf_tokenizer, hf_model), noop)
dblock = DataBlock(blocks=blocks, get_x=ColReader('de'), get_y=ColReader('en'), splitter=RandomSplitter())

Two lines!  Notice we pass in `noop` for our targets (e.g. our summaries) because the batch transform will take care of both out inputs and targets.

In [ ]:
# dblock.summary(wmt_df)

In [ ]:
dls = dblock.dataloaders(wmt_df, bs=4)

In [ ]:
b = dls.one_batch()

In [ ]:
len(b), b[0]['input_ids'].shape, b[1].shape

(2, torch.Size([4, 483]), torch.Size([4, 116]))

In [ ]:
dls.show_batch(dataloaders=dls, max_n=2, input_trunc_at=250, target_trunc_at=250)

,text,target
0,"<s> Was nun die Ergebnisse der Verhandlungen über die Anwendung der Artikel 3, 4, 5, 6 und 12 des Interimsabkommens bezüglich Warenhandel, öffentlicher Aufträge, Wettbewerb, Konsultationsmechanismen bei Fragen des geistigen Eigentums und Beilegung vo","Although for certain sectors there may be flaws - I am thinking specifically of the textiles sector, where the rules of origin issue causes great concern - the effects will be beneficial for both the European Union and Mexico. For the European Union"
1,"<s> Aufgrund der Interessenvielfalt und der durch latente Spannungen gekennzeichneten Situationen, um die es bei der Mittelmeerpolitik geht, muß diese verschiedene Aspekte umfassen: die Entwicklung friedenserhaltender Maßnahmen, nachdem nun der Fried","The Mediterranean policy is set in a context of wide-ranging interests and latent tensions, and it therefore has to include various different elements such as the development of peace-keeping initiatives - now that the Middle East peace process has"


## Tests

The purpose of the following tests is to ensure as much as possible, that the core DataBlock code above works for the pretrained **translation models** below.  These tests are excluded from the CI workflow because of how long they would take to run and the amount of data that would be required to download.

**Note**: Feel free to modify the code below to test whatever pretrained translation models you are working with ... and if any of your pretrained summarization models fail, please submit a github issue *(or a PR if you'd like to fix it yourself)*

In [ ]:
[ model_type for model_type in BLURR.get_models(task='ConditionalGeneration') 
 if (not model_type.__name__.startswith('TF')) ]

[transformers.models.bart.modeling_bart.BartForConditionalGeneration,
 transformers.models.bigbird_pegasus.modeling_bigbird_pegasus.BigBirdPegasusForConditionalGeneration,
 transformers.models.blenderbot.modeling_blenderbot.BlenderbotForConditionalGeneration,
 transformers.models.blenderbot_small.modeling_blenderbot_small.BlenderbotSmallForConditionalGeneration,
 transformers.models.fsmt.modeling_fsmt.FSMTForConditionalGeneration,
 transformers.models.led.modeling_led.LEDForConditionalGeneration,
 transformers.models.m2m_100.modeling_m2m_100.M2M100ForConditionalGeneration,
 transformers.models.mbart.modeling_mbart.MBartForConditionalGeneration,
 transformers.models.mt5.modeling_mt5.MT5ForConditionalGeneration,
 transformers.models.pegasus.modeling_pegasus.PegasusForConditionalGeneration,
 transformers.models.prophetnet.modeling_prophetnet.ProphetNetForConditionalGeneration,
 transformers.models.speech_to_text.modeling_speech_to_text.Speech2TextForConditionalGeneration,
 transformers.mo

In [ ]:
pretrained_model_names = [
    'facebook/bart-base',
    'facebook/wmt19-de-en',                      # FSMT
    'Helsinki-NLP/opus-mt-de-en',                # MarianMT
    'sshleifer/tiny-mbart',
    'google/mt5-small',
    't5-small'
]

In [ ]:
path = Path('./')
wmt_df = pd.DataFrame(ds['translation'], columns=['de', 'en'])

In [ ]:
#slow
#hide_output
model_cls = AutoModelForSeq2SeqLM
bsz = 2
seq_sz = 128
trg_seq_sz = 128

test_results = []
for model_name in pretrained_model_names:
    error=None
    
    print(f'=== {model_name} ===\n')
    
    hf_tok_kwargs = {}
    if (model_name == 'sshleifer/tiny-mbart'):
        hf_tok_kwargs['src_lang'], hf_tok_kwargs['tgt_lang'] = "de_DE", "en_XX"
            
    hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(model_name, 
                                                                      model_cls=model_cls, 
                                                                      tokenizer_kwargs=hf_tok_kwargs)
    
    print(f'architecture:\t{hf_arch}\ntokenizer:\t{type(hf_tokenizer).__name__}\n')
    
    # not all architectures include a native pad_token (e.g., gpt2, ctrl, etc...), so we add one here
    if (hf_tokenizer.pad_token is None): 
        hf_tokenizer.add_special_tokens({'pad_token': '<pad>'})  
        hf_config.pad_token_id = hf_tokenizer.get_vocab()['<pad>']
        hf_model.resize_token_embeddings(len(hf_tokenizer)) 
    
    before_batch_tfm = HF_Seq2SeqBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model,
                                                      padding='max_length', 
                                                      max_length=seq_sz,
                                                      max_target_length=trg_seq_sz)
    
    def add_t5_prefix(inp): return f'translate German to English: {inp}' if (hf_arch == 't5') else inp
    
    blocks = (HF_Seq2SeqBlock(before_batch_tfm=before_batch_tfm), noop)
    dblock = DataBlock(blocks=blocks, 
                   get_x=Pipeline([ColReader('de'), add_t5_prefix]), 
                   get_y=ColReader('en'), 
                   splitter=RandomSplitter())

    dls = dblock.dataloaders(wmt_df, bs=bsz) 
    b = dls.one_batch()
    
    try:
        print('*** TESTING DataLoaders ***\n')
        test_eq(len(b), 2)
        test_eq(len(b[0]['input_ids']), bsz)
        test_eq(b[0]['input_ids'].shape, torch.Size([bsz, seq_sz]))
        test_eq(len(b[1]), bsz)
        test_eq(b[1].shape, torch.Size([bsz, trg_seq_sz]))

        if (hasattr(hf_tokenizer, 'add_prefix_space')):
             test_eq(hf_tokenizer.add_prefix_space, True)
            
        test_results.append((hf_arch, type(hf_tokenizer).__name__, model_name, 'PASSED', ''))
        dls.show_batch(dataloaders=dls, max_n=2, input_trunc_at=1000)
        
    except Exception as err:
        test_results.append((hf_arch, type(hf_tokenizer).__name__, model_name, 'FAILED', err))

=== facebook/bart-base ===

architecture:	bart
tokenizer:	BartTokenizerFast

*** TESTING DataLoaders ***



,text,target
0,"<s> Was nun die Ergebnisse der Verhandlungen über die Anwendung der Artikel 3, 4, 5, 6 und 12 des Interimsabkommens bezüglich Warenhandel, öffentlicher Aufträge, Wettbewerb, Konsultationsmechanismen bei Fragen des geistigen Eigentums und Beilegung von Streitigkeiten betrifft, so gilt festzuhalten, daß wir das vorgesehene Maß</s>","Although for certain sectors there may be flaws - I am thinking specifically of the textiles sector, where the rules of origin issue causes great concern - the effects will be beneficial for both the European Union and Mexico. For the European Union, because the establishment of a free trade area will enable it to rebuild the presence Member States had in Mexican markets before the NAFTA agreement came into force, involving new expansion opportunities for European companies."
1,"<s> Es steht nunmehr fest, daß zur Erreichung dieses Ziels in den ländlichen Gebieten Arbeitsplätze geschaffen werden müssen, die nicht mehr nur mit der - für die Erhaltung der Umwelt und der Artenvielfalt gleichwohl wichtigen - Landwirtschaft im traditionellen Sinne in Zusammenhang stehen, sondern auch in den Bereichen ländlicher Tourismus, Sport und K</s>","It is now a fact that in order to attain this goal, rural areas need to see the creation of new jobs, which are no longer solely connected with agriculture, in the traditional sense - although this sector is important for the protection of the environment and the preservation of biodiversity - but also, for instance, in rural tourism, sport and cultural activities, environmental projects, SMEs, services, and so on."


=== facebook/wmt19-de-en ===

architecture:	fsmt
tokenizer:	FSMTTokenizer

*** TESTING DataLoaders ***



,text,target
0,"Was nun die Ergebnisse der Verhandlungen über die Anwendung der Artikel 3, 4, 5, 6 und 12 des Interimsabkommens bezüglich Warenhandel, öffentlicher Aufträge, Wettbewerb, Konsultationsmechanismen bei Fragen des geistigen Eigentums und Beilegung von Streitigkeiten betrifft, so gilt festzuhalten, daß wir das vorgesehene Maßnahmenpaket für ein gutes Abkommen halten, auch wenn es für bestimmte Sektoren mit Nachteilen verbunden ist - ich denke im einzelnen an die Textilindustrie, die sich insbesondere wegen der Ursprungsregeln Sorgen macht -, ein Abkommen, das sowohl der Europäischen Union als auch Mexiko </s>","Although for certain sectors there may be flaws - I am thinking specifically of the textiles sector, where the rules of origin issue causes great concern - the effects will be beneficial for both the European Union and Mexico. For the European Union, because the establishment of a free trade area will enable it to rebuild the presence Member States had in Mexican markets before the NAFTA agreement came into force, involving new expansion opportunities for European companies."
1,"Deshalb ist unser Ausschuß der Auffassung, daß der Vorschlag für eine Richtlinie über die gemeinschaftlichen aufsichtsrechtlichen Regelungen für die Pensionsfonds baldmöglichst vorgelegt und angenommen werden sollte, wobei unsere heutige Debatte zu berücksichtigen ist und ernsthafte Garantien für die externe und interne Kontrolle der jeweiligen Fonds sowie die Berechnung ihrer Aufwendungen für Risikovorsorge gegeben werden müssen. </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>","This is why our Committee feels that the proposal for a directive on Community prudential regulations applying to pension funds must be tabled and adopted as quickly as possible, bearing in mind our discussion today, with proper guarantees for the external and internal supervision of the funds concerned and for the calculation of their prudential commitment."


=== Helsinki-NLP/opus-mt-de-en ===

architecture:	marian
tokenizer:	MarianTokenizer

*** TESTING DataLoaders ***



...

,text,target
0,"Was nun die Ergebnisse der Verhandlungen über die Anwendung der Artikel 3, 4, 5, 6 und 12 des Interimsabkommens bezüglich Warenhandel, öffentlicher Aufträge, Wettbewerb, Konsultationsmechanismen bei Fragen des geistigen Eigentums und Beilegung von Streitigkeiten betrifft, so gilt festzuhalten, daß wir das vorgesehene Maßnahmenpaket für ein gutes Abkommen halten, auch wenn es für bestimmte Sektoren mit Nachteilen verbunden ist - ich denke im einzelnen an die Textilindustrie, die sich insbesondere wegen der Ursprungsregeln Sorgen macht -, ein Abkommen, das sowohl der Europäischen</s>de_DE","Although for certain sectors there may be flaws - I am thinking specifically of the textiles sector, where the rules of origin issue causes great concern - the effects will be beneficial for both the European Union and Mexico. For the European Union, because the establishment of a free trade area will enable it to rebuild the presence Member States had in Mexican markets before the NAFTA agreement came into force, involving new expansion opportunities for European companies."
1,"Diese Empfehlungen, die weitgehend mit den Orientierungen des Programmdokuments der portugiesischen Ratspräsidentschaft übereinstimmen und demzufolge auf dem Gipfel in Lissabon behandelt werden, wurden im Ausschuß für Wirtschaft und Währung dieses Parlaments abgelehnt oder verzerrt, und zwar von einer Mehrheit, die ihre zerbrechliche Einheit einzig und allein in der ideologischen Abneigung gegen eine Europäische Union, die auf der Konvergenz der einzelstaatlichen Politikmaßnahmen anstatt nur auf einem Marktgefüge beruht, zu finden vermochte.</s>de_DE<pad><pad><pad><pad>","These recommendations, which, for the most part, are consistent with the guidelines in the Portuguese presidency' s work programme and which will therefore be discussed at the Lisbon Summit, have been rejected or distorted by the Committee on Economic and Monetary Affairs of this Parliament, by a majority which only managed to achieve a fragile unity in its rejection of the idea of a European Union which is founded on the convergence of national policies and not just on the grouping together of markets."


=== google/mt5-small ===

architecture:	mt5
tokenizer:	T5TokenizerFast

*** TESTING DataLoaders ***



,text,target
0,"Was nun die Ergebnisse der Verhandlungen über die Anwendung der Artikel 3, 4, 5, 6 und 12 des Interimsabkommens bezüglich Warenhandel, öffentlicher Aufträge, Wettbewerb, Konsultationsmechanismen bei Fragen des geistigen Eigentums und Beilegung von Streitigkeiten betrifft, so gilt festzuhalten, daß wir das vorgesehene Maßnahmenpaket für ein gutes Abkommen halten, auch wenn es für bestimmte Sektoren mit Nachteilen verbunden ist - ich denke im einzelnen an die Textilindustrie, die sich insbesonder</s>","Although for certain sectors there may be flaws - I am thinking specifically of the textiles sector, where the rules of origin issue causes great concern - the effects will be beneficial for both the European Union and Mexico. For the European Union, because the establishment of a free trade area will enable it to rebuild the presence Member States had in Mexican markets before the NAFTA agreement came into force, involving new expansion opportunities for European companies."
1,"Sollte diese Initiative nicht zum Erfolg führen, wird meine Fraktion auf der Grundlage desselben Artikels eine mündliche Anfrage an den Rat stellen, damit dieser die Sorgen und Anliegen dieses Parlaments wirklich ernst nimmt und die Bestimmungen der Verträge in der Praxis einhält, nämlich das Parlament zu allen jenen Aspekten, die die Bürger der Europäischen Union betreffen und beunruhigen, informiert und in die Arbeit einbindet.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>","If this initiative is not successful, my political group will present, pursuant to the same article, an oral question to the Council so that the Council really takes seriously the worries and concerns of this Parliament and really does what is laid down in the Treaties, that is to say, to inform and cooperate with Parliament in all those areas which affect and concern the citizens of the European Union."


=== t5-small ===

architecture:	t5
tokenizer:	T5TokenizerFast

*** TESTING DataLoaders ***



,text,target
0,"translate German to English: Was nun die Ergebnisse der Verhandlungen über die Anwendung der Artikel 3, 4, 5, 6 und 12 des Interimsabkommens bezüglich Warenhandel, öffentlicher Aufträge, Wettbewerb, Konsultationsmechanismen bei Fragen des geistigen Eigentums und Beilegung von Streitigkeiten betrifft, so gilt festzuhalten, daß wir das vorgesehene Maßnahmenpaket für ein gutes Abkommen halten, auch wenn es für bestimmte Sektoren mit Nachteilen verbunden ist - ich denke im einzelnen an die Textilindustrie,</s>","Although for certain sectors there may be flaws - I am thinking specifically of the textiles sector, where the rules of origin issue causes great concern - the effects will be beneficial for both the European Union and Mexico. For the European Union, because the establishment of a free trade area will enable it to rebuild the presence Member States had in Mexican markets before the NAFTA agreement came into force, involving new expansion opportunities for European companies."
1,"translate German to English: Nach der Tagesordnung folgt die Empfehlung für die zweite Lesung (A5-0047/2000) im Namen des Ausschusses für Umweltfragen, Volksgesundheit und Verbraucherpolitik betreffend den gemeinsamen Standpunkt des Rates im Hinblick auf den Erlaß einer Richtlinie des Europäischen Parlaments und des Rates über Kakao- und Schokoladeerzeugnisse für die menschliche Ernährung (9947/1/1999 - C5-0252/1999 - 1996/0112(COD)) (Berichterstatter: Abgeordneter Lannoy</s>","The next item is the recommendation for second reading (A5-0047/2000), on behalf of the Committee on the Environment, Public Health and Consumer Policy, on the common position adopted by the Council with a view to adopting a European Parliament and Council Directive relating to cocoa and chocolate products intended for human consumption (9947/1/1999 - C5-0252/1999 - 1996/0112(COD)) (Rapporteur: Mr Lannoye)."


In [ ]:
#slow
#hide_input
test_results_df = pd.DataFrame(test_results, columns=['arch', 'tokenizer', 'model_name', 'result', 'error'])
display_df(test_results_df)

,arch,tokenizer,model_name,result,error
0,bart,BartTokenizerFast,facebook/bart-base,PASSED,
1,fsmt,FSMTTokenizer,facebook/wmt19-de-en,PASSED,
2,marian,MarianTokenizer,Helsinki-NLP/opus-mt-de-en,PASSED,
3,mbart,MBartTokenizerFast,sshleifer/tiny-mbart,PASSED,
4,mt5,T5TokenizerFast,google/mt5-small,PASSED,
5,t5,T5TokenizerFast,t5-small,PASSED,


## Cleanup

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_data-core.ipynb.
Converted 01_modeling-core.ipynb.
Converted 02_data-language-modeling.ipynb.
Converted 02_modeling-language-modeling.ipynb.
Converted 03_data-token-classification.ipynb.
Converted 03_modeling-token-classification.ipynb.
Converted 04_data-question-answering.ipynb.
Converted 04_modeling-question-answering.ipynb.
Converted 10_data-seq2seq-core.ipynb.
Converted 10_modeling-seq2seq-core.ipynb.
Converted 11_data-seq2seq-summarization.ipynb.
Converted 11_modeling-seq2seq-summarization.ipynb.
Converted 12_data-seq2seq-translation.ipynb.
Converted 12_modeling-seq2seq-translation.ipynb.
Converted 99a_examples-multilabel.ipynb.
Converted index.ipynb.
